In [1]:
import pandas as pd

In [2]:
df=pd.read_csv(r"E:\Cyberbullying\dataset\raw\kurrek.2020.slur-corpus.csv")

In [3]:
import pandas as pd

def load_kurrek_data():
    # Load dataset
    df = pd.read_csv(r"E:\Cyberbullying\dataset\raw\kurrek.2020.slur-corpus.csv")

    # Drop noise
    df = df[df['gold_label'] != 'CMP']

    # Map labels to binary
    label_map = {
        'DEG': '1',
        'APR': '1',
        'NDG': '0',
        'HOM': '0'
    }
    df['binary_label'] = df['gold_label'].map(label_map)

    # Optional: Drop rows with NaN (if any remain)
    df = df.dropna(subset=['binary_label', 'body'])

    return df[['body', 'binary_label']]


In [4]:
load_kurrek_data()

,body,binary_label
0,Fuck that I dont wanna watch tranny porn.,1
1,Opie just wanted to have a good time. The same...,1
2,Fuck that faggot Fallon. Fucking sissy boy act...,1
3,BbBB...b.b..b.bb but OP's a faggot,1
4,Who even uses the word tranny except for trans...,0
...,...,...
39998,*As an alcoholic who on occasion has one too m...,1
39999,"It's not inaccurate, it's indoctrination of yo...",1
40000,What did you expect?? when they banned /r/nigg...,0
40001,That was hilariously bad. They used the insult...,0


In [5]:
TOXIC_PATH= r"E:\Cyberbullying\dataset\new\OFF_HATE_TOXIC_DATASET.json"

In [6]:
def load_toxic_dataset(path=TOXIC_PATH):
    # Load dataset
    df = pd.read_json(path)

    # OFF_HATEFUL_TOXIC: offensive, hateful, or toxic content
    # NOT_OFF_HATEFUL_TOXIC: non-offensive, non-hateful, or non-toxic content
    df['label'] = df['label'].map({
        'NOT_OFF_HATEFUL_TOXIC': 0,
        'OFF_HATEFUL_TOXIC': 1})
    df = df.dropna(subset=['text', 'label'])
    df = df.drop_duplicates(subset=['text', 'label'], keep='first')
    return df[['text', 'label']]
toxic_df=load_toxic_dataset()

In [7]:
from sklearn.utils import resample

def balance_dataset(toxic_df,name):
    df_majority = toxic_df[toxic_df['label'] == 0]
    df_minority = toxic_df[toxic_df['label'] == 1]

    # Downsample majority class
    df_majority_downsampled = resample(df_majority,
                                    replace=False,     # without replacement
                                    n_samples=len(df_minority),  # match minority count
                                    random_state=42)

    # Combine balanced data
    df_balanced = pd.concat([df_majority_downsampled, df_minority])

    # Shuffle
    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

    print(df_balanced['label'].value_counts())
    df_balanced.to_csv(f"E:\Cyberbullying\dataset\preprocessed\{name}.csv", index=False)

<string>:20: SyntaxWarning: invalid escape sequence '\{'
<>:20: SyntaxWarning: invalid escape sequence '\{'
<>:20: SyntaxWarning: invalid escape sequence '\C'
<string>:20: SyntaxWarning: invalid escape sequence '\{'
<>:20: SyntaxWarning: invalid escape sequence '\{'
<>:20: SyntaxWarning: invalid escape sequence '\C'
C:\Users\user\AppData\Local\Temp\ipykernel_19560\3887051702.py:20: SyntaxWarning: invalid escape sequence '\{'
  df_balanced.to_csv(f"E:\Cyberbullying\dataset\preprocessed\{name}.csv", index=False)
C:\Users\user\AppData\Local\Temp\ipykernel_19560\3887051702.py:20: SyntaxWarning: invalid escape sequence '\C'
  df_balanced.to_csv(f"E:\Cyberbullying\dataset\preprocessed\{name}.csv", index=False)


In [8]:
balance_dataset(toxic_df,'OFF_HATE_TOXIC_DATASET')

label
0    19413
1    19413
Name: count, dtype: int64


In [9]:
GHS_PATH = r"E:\Cyberbullying\dataset\new\gender-hate-speech-train.csv"

In [10]:
def load_ghs_dataset(path=GHS_PATH):
    # Load dataset
    ghs_df = pd.read_csv(path)

    ghs_df['Label'] = ghs_df['Label'].apply(lambda x: 0 if x ==0 else 1) #


    # Optional: Drop rows with NaN (if any remain)
    ghs_df = ghs_df.dropna(subset=['Text', 'Label'])
    ghs_df = ghs_df.drop_duplicates(subset=['Text', 'Label'], keep='first')
    ghs_df.rename(columns={'Label': 'label'}, inplace=True) # Align column name
    ghs_df.rename(columns={'Text': 'text'}, inplace=True) # Align column name

    return ghs_df[['text', 'label']]

In [11]:
ghs_df = load_ghs_dataset()

In [12]:
balance_dataset(ghs_df,'GHS_DATASET')

label
0    7011
1    7011
Name: count, dtype: int64


In [13]:
MULTI_HATE_PATH=r"E:\Cyberbullying\dataset\new\hate-multi.parquet"

In [14]:
def load_multi_hate_dataset(path=MULTI_HATE_PATH):
    # Load dataset
    multi_df = pd.read_parquet(path)


    
    multi_df = multi_df.dropna(subset=['text', 'label'])
    multi_df = multi_df.drop_duplicates(subset=['text', 'label'], keep='first')
    
    return multi_df[['text', 'label']]

In [15]:
multi_df=load_multi_hate_dataset()

In [16]:
multi_df['label'].value_counts()

label
0    44826
1    17515
Name: count, dtype: int64

In [17]:
balance_dataset(multi_df,'MULTI_HATE_DATASET')

label
0    17515
1    17515
Name: count, dtype: int64


In [ ]:
import pandas as pd

# Load datasets
df1 = pd.read_csv(r'E:\Cyberbullying\dataset\preprocessed\GHS_DATASET.csv', names=['text', 'label'], header=None)
df2 = pd.read_csv(r"E:\Cyberbullying\dataset\preprocessed\MULTI_HATE_DATASET.csv", names=['text', 'label'], header=None)
df3 = pd.read_csv(r"E:\Cyberbullying\dataset\preprocessed\OFF_HATE_TOXIC_DATASET.csv", names=['text', 'label'], header=None)

# Combine them
merged_df = pd.concat([df1, df2, df3], ignore_index=True)

# Remove duplicate rows (optional but common)
merged_df = merged_df.drop_duplicates(subset=['text', 'label']).reset_index(drop=True)

# Check result
print(merged_df.shape)
print(merged_df['label'].value_counts())
merged_df.to_csv(r"E:\Cyberbullying\dataset\preprocessed\merged_dataset.csv", index=False)
print("Merged dataset saved successfully.")

KeyError: 'text'

In [22]:
merged_df

,text,label
0,text,label
1,like this if you wanna kiss princess bubblegum...,0
2,Forbidding women from winning because you don'...,0
3,Dreamt there was a new lesbian movie on Netfli...,0
4,I have no interest in speaking with anyone fal...,1
...,...,...
87873,Ever hear him praise General Lee ? Has he ev...,0
87874,@user @user Thank you. Not all men are bad. In...,0
87875,I do not call him IslamaObama for nothing he i...,1
87876,💜💓💜 Happy Wednesday My Dear #GabFam Friends! 💜...,0


In [ ]:
import pandas as pd
OMG_PATH= r"E:\Cyberbullying\dataset\preprocessed\OMG.csv"


In [6]:
load_omg_dataset()

,text,label
0,like this if you wanna kiss princess bubblegum...,0
1,Forbidding women from winning because you don'...,0
2,Dreamt there was a new lesbian movie on Netfli...,0
3,I have no interest in speaking with anyone fal...,1
4,trevor has some sort of gender and its not cis...,0
...,...,...
87872,Ever hear him praise General Lee ? Has he ev...,0
87873,@user @user Thank you. Not all men are bad. In...,0
87874,I do not call him IslamaObama for nothing he i...,1
87875,💜💓💜 Happy Wednesday My Dear #GabFam Friends! 💜...,0
